In [1]:
from z3 import *

#### Programa
Tentar resolver o problema com INDUÇÃO (não usar lookahead) usando verificação de triplos de Hoare em ciclos

```python
assume m >= 0 and n >= 0 and r == 0 and x == m and y == n 
0: while y > 0: 
      # inv y>=0 and x*y+r == m*n
1:    if y & 1 == 1: 
          y , r = y-1 , r+x 
2:    x , y = x<<1 , y>>1 
3: assert r == m * n 
```

# 1. Provar que o programa termina

O variante do programa pode ser definido como:

$$ V \equiv y $$

Utilizando $k$-indução queremos então provar, para um dado traço $\alpha = \{ \alpha_i \; | \; i = 0, 1, \dots, | \alpha |-1 \}$, que o programa termina (ou seja, a variável $\mathtt{pc}$ toma o valor $3$). Para tal ocorrer, as seguintes propriedades têm de ser verificadas:
    
- Positivo: 
$$ \forall_{i}. \alpha_i \in \alpha, \; V_i \ge 0$$

- Decrescente:
$$ \forall_{i}. \alpha_i \in \left( \alpha \setminus \alpha_{|\alpha|-1} \right), \; V_{i+1} < V_i $$

- Útil:
$$ V = 0 \rightarrow \left( \; \mathtt{pc} = 0 \wedge y \le 0 \; \right) $$

In [2]:
from z3 import *

def induction_always(declare, init, trans, var, prop, l):
    # Declarar o traço
    solver = Solver()
    trace = {i: declare(i) for i in range(2)}
    
    # Testar caso de base
    solver.add(init(trace[0]))
    if prop == "decreases":
        solver.add(Not(var(trace[0], trans, l)))
    else:
        solver.add(Not(var(trace[0])))
    
    if solver.check() == sat:
        print("Induction breaks down in the initial trace.")
        m = solver.model()
        
        for v in trace[0]:
            print(v, "=", m[trace[0][v]])
        return
    elif solver.check() != unsat:
        return

    # Testar caso indutivo
    solver = Solver()
    solver.add(init(trace[0]))
    
    if prop == "decreases":
        solver.add(var(trace[0], trans, l))
    else:
        solver.add(var(trace[0]))
    solver.add(trans(trace[0], trace[1]))
    if prop == "decreases":
        solver.add(Not(var(trace[1], trans, l)))
    else:
        solver.add(Not(var(trace[1])))
    
    if solver.check() == sat:
        print("Induction breaks down in the inductive trace.")
        m = solver.model()
        
        for v in trace[0]:
            print(v, "=", m[trace[0][v]])
        return
    elif solver.check() == unsat:
        print(f"The property \"{prop}\" holds.")

In [3]:
def declare(i):
    trace = {}
    trace["x"] = BitVec(f"x_{i}", 16)
    trace["y"] = BitVec(f"y_{i}", 16)
    trace["r"] = BitVec(f"r_{i}", 16)
    trace["m"] = BitVec(f"m_{i}", 16)
    trace["n"] = BitVec(f"n_{i}", 16)
    trace["pc"] = BitVec(f"pc_{i}", 16)
    
    return trace

def init(trace):
    r1 = And(trace["pc"]==0)
    r2 = And(trace["r"]==0, trace["m"]>0, trace["n"]>0, trace["x"]==trace["m"], trace["y"]==trace["n"])
    return And(r1, r2)

def trans(prev, curr):
    # Condições para pc == 0
    cond1_pc0 = And(prev["pc"]==0, prev["y"]>0, curr["x"]==prev["x"], curr["y"]==prev["y"],
                    curr["m"]==prev["m"], curr["n"]==prev["n"], curr["r"]==prev["r"],
                    curr["pc"]==1)
    cond2_pc0 = And(prev["pc"]==0, Not(prev["y"]>0), curr["x"]==prev["x"], curr["y"]==prev["y"],
                    curr["m"]==prev["m"], curr["n"]==prev["n"], curr["r"]==prev["r"],
                    curr["pc"]==3)
    cond_pc0 = Or(cond1_pc0, cond2_pc0)
    
    # Condições para pc == 1
    cond1_pc1 = And(prev["pc"]==1, prev["y"]&1==1, curr["x"]==prev["x"], curr["y"]==prev["y"]-1,
                    curr["m"]==prev["m"], curr["n"]==prev["n"], curr["r"]==prev["r"]+prev["x"],
                    curr["pc"]==2)
    cond2_pc1 = And(prev["pc"]==1, Not(prev["y"]&1==1), curr["x"]==prev["x"], curr["y"]==prev["y"],
                    curr["m"]==prev["m"], curr["n"]==prev["n"], curr["r"]==prev["r"],
                    curr["pc"]==2)
    cond_pc1 = Or(cond1_pc1, cond2_pc1)
    
    # Condições para pc == 2
    cond_pc2 = And(prev["pc"]==2, curr["x"]==prev["x"]<<1, curr["y"]==prev["y"]>>1,
                   curr["m"]==prev["m"], curr["n"]==prev["n"], curr["r"]==prev["r"],
                   curr["pc"]==0)
    
    # Condições para pc == 3
    cond_pc3 = And(prev["pc"]==3, curr["x"]==prev["x"], curr["y"]==prev["y"],
                   curr["m"]==prev["m"], curr["n"]==prev["n"], curr["r"]==prev["r"],
                   curr["pc"]==prev["pc"])
    
    return Or(cond_pc0, cond_pc1, cond_pc2, cond_pc3)

In [4]:
def trans_(prev, curr):
    # Condições para pc == 0
    cycle1 = And(prev["y"]&1!=1, curr["y"]==prev["y"]>>1, curr["x"]==prev["x"]<<1,
                 curr["r"]==prev["r"])
    cycle2 = And(prev["y"]&1==1, curr["y"]==(prev["y"]-1)>>1, curr["r"]==prev["r"]+prev["x"],
                 curr["x"]==prev["x"]<<1)
    cond1_pc0 = And(prev["pc"]==0, curr["m"]==prev["m"], curr["n"]==prev["n"],
                    Or(cycle1, cycle2), curr["pc"]==0)
    cond2_pc0 = And(prev["pc"]==0, Not(prev["y"]>0), curr["x"]==prev["x"], curr["y"]==prev["y"],
                    curr["m"]==prev["m"], curr["n"]==prev["n"], curr["r"]==prev["r"],
                    curr["pc"]==3)
    cond_pc0 = Or(cond1_pc0, cond2_pc0)
    
    # Condições para pc == 3
    cond_pc3 = And(prev["pc"]==3, curr["x"]==prev["x"], curr["y"]==prev["y"],
                   curr["m"]==prev["m"], curr["n"]==prev["n"], curr["r"]==prev["r"],
                   curr["pc"]==prev["pc"])
    
    return Or(cond_pc0, cond_pc3)

In [5]:
def variant(trace):
    return trace["y"]

def var_positive(trace):
    return variant(trace)>=0

def var_decreases(trace, trans, l=3):
    traces = {i: declare(-i) for i in range(1, l+1)}
    c1 = And([trans(traces[i], traces[i+1]) for i in range(1, l)] + [trans(trace, traces[1])])
    c2 = Or(variant(traces[l])<variant(trace), variant(traces[l])==0)
    r = ForAll(list(traces[l].values()), Implies(c1, c2))
    return r

def var_useful(trace):
    return Implies(variant(trace)==0, And(trace["pc"]==0, trace["y"]<=0))

### Prova com Lookahead

In [6]:
induction_always(declare, init, trans, var_positive, "positive", 3)
induction_always(declare, init, trans, var_decreases, "decreases", 3)
induction_always(declare, init, trans, var_useful, "useful", 3)

The property "positive" holds.
The property "decreases" holds.
The property "useful" holds.


### Prova com Indução

In [7]:
induction_always(declare, init, trans_, var_positive, "positive", 1)
induction_always(declare, init, trans_, var_decreases, "decreases", 1)
induction_always(declare, init, trans_, var_useful, "useful", 1)

The property "positive" holds.
The property "decreases" holds.
The property "useful" holds.


# 2. Correção Parcial

## WPC: Havoc

$$\lbrack\,\mathsf{P}\,\rbrack\;\equiv\; \phi\,\to\,\bigwedge\,a\,\centerdot\,(b[\upsilon/a]\,\to\,[\,S\,][\upsilon/a])\,\land\,(\neg b[\upsilon/a] \,\to\,\varphi[\upsilon/a])$$

$$ \phi \; \equiv \; m \ge 0 \wedge n \ge 0 \wedge x = m \wedge y = n \wedge r=0 $$
$$ \varphi \; \equiv \; r = m \cdot n $$
$$ b \; \equiv \; y > 0 $$

```python
1: if y & 1 == 1: y , r = y-1 , r+x
2: x , y = x<<1 , y>>1
```

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = True \\
[{\sf assert}\:\phi] = \phi \\
[ x = e ] = True \\
[(C_1 || C_2)] = [C_1] \wedge [C_2] \\
\\
[{\sf skip}\, ; C] = [C] \\
[{\sf assume}\:\phi\, ; C] = \phi \to [C] \\
[{\sf assert}\:\phi\, ; C] = \phi \wedge [C] \\
[ x = e \, ; C] = [C][e/x] \\
[(C_1 || C_2)\, ; C] = [(C_1 ; C) || (C_2 ; C)]
\end{array}
$

$$ [\,S\,] \equiv \mathtt{assume} \; y \% 1 = 1; \; y = y-1; \; r = r+x; \; x = x \ll 1 ; \; y = y \gg 1; \; || \; \mathtt{assume} \; y \% 1 \neq 1 ;\; x = x \ll 1 ; \; y = y \gg 1; \; $$

$$ [\,S\,] \equiv \mathtt{True} $$

$$\lbrack\,\mathsf{P}\,\rbrack\;\equiv\; \phi\,\to\,\bigwedge\,a\,\centerdot\,(\neg b[\upsilon/a] \,\to\,\varphi[\upsilon/a])$$

In [9]:
def pre(trace):
    r1 = And(trace["m"]>=0, trace["n"]>=0)
    r2 = And(trace["y"]==trace["n"], trace["x"]==trace["m"], trace["r"]==0)
    return And(r1, r2)

pos = lambda trace: trace["r"] == trace["m"]*trace["n"]
b = lambda trace: trace["y"] > 0

def havoc(declare, pre, pos, b, S):
    trace = declare(0)
    solver = Solver()
    solver.add(Not(Implies(pre(trace), Implies(Not(b(trace)), pos(trace)))))
    
    if solver.check() == sat:
        print("O programa não está correto.")
    else:
        print("O programa está correto.")
    
havoc(declare, pre, pos, b, True)

O programa está correto.


## SPC: Unfold

In [10]:
from math import log
log(2**15 - 1, 2)

14.999955971769559

In [15]:
def bmc_unfold(declare, init, trans, pos, n):
    trace = {i: declare(i) for i in range(n)}
    solver = Solver()
    solver.add(init(trace[0]))
    for i in range(n-1):
        solver.add(trans(trace[i], trace[i+1]))
    solver.add(Not(pos(trace[n-1])))
    
    if solver.check() == sat:
        print("O programa está incorreto.")
        m = solver.model()
        
        for v in trace[0]:
            print(v, "=", m[trace[0][v]])
    else:
        print("O programa está correto.")
        
bmc_unfold(declare, init, trans_, pos, 16)

O programa está correto.
